# This Document was build for Logistic Regression Analysis This model could be used for 2 class + 
# <font color='firebrick'> Author: Rick Ou
# <font color='firebrick'>  Version: 1.0.1
Updated 3/22/2021
Package Used: Panda, sklearn, statsmodels
(Supervised Classification Model)

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
smarket = pd.read_csv('smarket.csv')
smarket.head()

,Year,Lag1,Lag2,Lag3,Lag4,Lag5,Volume,Today,Direction
0,2001,0.381,-0.192,-2.624,-1.055,5.010,1.1913,0.959,Up
1,2001,0.959,0.381,-0.192,-2.624,-1.055,1.2965,1.032,Up
2,2001,1.032,0.959,0.381,-0.192,-2.624,1.4112,-0.623,Down
3,2001,-0.623,1.032,0.959,0.381,-0.192,1.2760,0.614,Up
4,2001,0.614,-0.623,1.032,0.959,0.381,1.2057,0.213,Up


### Using Stat package to fit into 2 - Class LogisticRegression 

In [2]:
import statsmodels.formula.api as smf

The prediction cant be text/qualitative so we need to turn the direction columns into Numerical:

In [3]:
smarket['Up'] = np.where(smarket['Direction'] == 'Up',1,0)

In [4]:
X = smarket[['Lag1', 'Lag2', 'Lag3', 'Lag4', 'Lag5','Volume']]
y = smarket['Up']
train_bool = smarket['Year'] < 2005
X_test = X[~train_bool]
y_test = y[~train_bool]

In [48]:
print("X_test.shape: ", X_test.shape)
print("y_test.shape: ", y_test.shape)

X_test.shape:  (252, 6)
y_test.shape:  (252,)


Those number associate with the possibility of going up since we set the direction UP

In [58]:
result7 = smf.logit('Up ~ Lag1 + Lag2 + Lag3 + Lag4 + Lag5 + Volume',
                    data=smarket, subset = train_bool).fit()
result7_prob = result7.predict(X_test)
result7_pred = (result7_prob > 0.5)
confusion_matrix(y_test, result7_pred)

Optimization terminated successfully.
         Current function value: 0.691936
         Iterations 4


array([[77, 34],
       [97, 44]])

In [52]:
#np.mean(result7_pred != y_test)
np.mean(result7_pred == y_test) 

0.4801587301587302

Building the model with only Lag1 and Lag2 to see if the result improvement 

In [15]:
result8 = smf.logit('Up ~ Lag1 + Lag2', data=smarket, subset = train_bool).fit()
result8_prob = result8.predict(X_test)
result8_pred = (result8_prob > 0.5)
confusion_matrix(y_test, result8_pred)

Optimization terminated successfully.
         Current function value: 0.692085
         Iterations 3


array([[ 35,  76],
       [ 35, 106]])

In [14]:
np.mean(result8_pred == y_test)

0.5595238095238095

In [10]:
(35+106)/(35+76+35+106)

0.5595238095238095

### Using Sklearn.model Package to fit into 2 + Class Logistic Regression

In [5]:
from sklearn.model_selection import train_test_split
df_wine = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
# if the Wine dataset is temporarily unavailable from the
# UCI machine learning repository, un-comment the following line
# of code to load the dataset from a local path:
# df_wine = pd.read_csv('wine.data', header=None)
df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                    'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values
X_train, X_test, y_train, y_test =\
    train_test_split(X, y,
                     test_size=0.55,
                     random_state=0,
                     stratify=y)

In [6]:
from sklearn.linear_model import LogisticRegression
mlr = LogisticRegression(penalty='none', max_iter=300).fit(X_train, y_train)
# If we set max_iter=100, it will throw out a warning that it failed to converge
print(mlr.predict(X_test[0:5, :]))
print(mlr.predict_proba(X_test[0:5, :]))
print(mlr.score(X_test, y_test))

[2 2 3 2 3]
[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 0. 1.]]
0.9693877551020408
